## 자동차 리콜 3개년 분석

## 데이터 출처
-  https://www.data.go.kr/data/3048950/fileData.do

[받은 파일]
- 2020년:자동차리콜현황(2020) v3.0.csv
- 2021년:자동차리콜현황(2021).csv
- 2022년:한국교통안전공단_자동차결함 리콜현황_20221231.csv

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_csv('./data/다운/자동차리콜현황(2020) v3.0.csv',encoding="euc-kr")
df2 = pd.read_csv('./data/다운/자동차리콜현황(2021).csv',encoding="euc-kr")
df3 = pd.read_csv('./data/다운/한국교통안전공단_자동차결함 리콜현황_20221231.csv',encoding="euc-kr")

In [ ]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
print("Before:", len(df))
df = df.drop_duplicates()
print("After:", len(df))

In [ ]:
df['생산기간(부터)'] = pd.to_datetime(df['생산기간(부터)'])
df['생산_년']=df['생산기간(부터)'].dt.year
df['생산_월']=df['생산기간(부터)'].dt.month
df['생산_일']=df['생산기간(부터)'].dt.day

df['생산기간(까지)'] = pd.to_datetime(df['생산기간(까지)'])
df['종료_년']=df['생산기간(까지)'].dt.year
df['종료_월']=df['생산기간(까지)'].dt.month
df['종료_일']=df['생산기간(까지)'].dt.day

df['리콜개시일'] = pd.to_datetime(df['리콜개시일'])
df['리콜_년']=df['리콜개시일'].dt.year
df['리콜_월']=df['리콜개시일'].dt.month
df['리콜_일']=df['리콜개시일'].dt.day

## 1. 제조사별 리콜 현황
* 3년동안 리콜 건수가 가장 많은 제조사는?

In [ ]:
df["제작자"].value_counts()

In [ ]:
tmp = df['제작자'].value_counts().to_frame('count')

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 6))
plt.rc('font', family = "Malgun Gothic") #한글사용
plt.rcParams['axes.unicode_minus']=False #마이너스 부호 깨짐

x = tmp.index
y = tmp['count']

plt.bar(x, height=y)
plt.xticks(rotation=90)
plt.title('제조사별 리콜 현황')
plt.show()

```
문제: 3년동안 리콜건수가 가장 많은 제조사는?
답: 비엠더블유코리아
```

## 2.(그나마) 튼튼하게 만드는 기업 vs  Worst 기업 top10

* 출고 년도부터 리콜 개시일 사이의 기간을 사용기간이라고 할때, 
* 이 사용기간이 길면 길수록 그나마 튼튼하게 만들었다고 볼 수 있다. 
* 물론, 여기 리콜현황에 등록되지 않은 제조사가 베스트이겠지만....
* 사용기간의 평균이 가장 높고 낮은 10개 기업 리스트를 알아보자

In [ ]:
# 사용기간 컬럼 새로 만들기(파생변수 생성해서)

df['사용기간'] = df['리콜개시일'] - df['생산기간(까지)']

In [ ]:
df.head(3)

In [ ]:
# 제조사별 사용기간이 가장 긴 순서대로 정렬
avg_usage = df.groupby('제작자')['사용기간'].mean().sort_values(ascending=False).to_frame('튼튼기간')


In [ ]:
avg_usage.head(3)

In [ ]:
avg_usage['튼튼기간'] = avg_usage['튼튼기간'].dt.days

In [ ]:
good_brand = avg_usage.head(10)
bad_brand = avg_usage.tail(10)

In [ ]:
good_brand['튼튼기간']

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.rc('font', family = "Malgun Gothic") #한글사용
plt.rcParams['axes.unicode_minus']=False #마이너스 부호 깨짐


x = good_brand.index
y = good_brand['튼튼기간']

plt.subplot(1, 2, 1)
plt.bar(x=x, height=y)
plt.xticks(rotation=90)
plt.title('사용기간 가장 긴 제조사(good)')

x = bad_brand.index
y = bad_brand['튼튼기간']

plt.subplot(1, 2, 2)
plt.bar(x=x, height=y)
plt.xticks(rotation=90)
plt.title('사용기간 가장 짧은 제조사(bad)')

plt.show()


```
문제:그나마 튼튼 제조사 top 3는?
답: 한신특장, 한국닛산, 비엠더블유코리아
```

## 3. 리콜이 가장 많은 제조사 1위의 리콜 원인은 ?
* 리콜 대마왕 XXX 기업의 문제의 모델 xxx에 대해 대규모 손해배상청구 소송을 진행하려 한다. 
* 이 회사의 주요 리콜 사유를 알아내고,(워드 클라우드로 제작해서 키워드 추출)
* 해당 사유로 리콜된 최고 문제 모델을 찾아내서
* 소송으로 가즈아~

In [ ]:
# 리콜건수가 가장 많은 제조사는?
brand = df['제작자'].value_counts().index[0]
brand

In [ ]:
# 리콜 사유는?
result = df[df['제작자'] == brand]
result.head(3)


In [ ]:
# 워드클라우드를 사용해 키워드 추출

from wordcloud import WordCloud, STOPWORDS
set(STOPWORDS)
spwords = set(['되었습니다','발견','되었습니다.',"수이기는","적은 수이기는","동안", "인하여", "있는", "경우", "있습니다", "가능성이", "않을", "차량의", "가", "에", "될", "이",
               "인해", "수", "중", "시", "또는", "있음", "의", "및", "있으며", "발생할", "이로", "오류로", "해당", "있어", "내에",
                "못", '으로", "의한', "위해", '후', '할', '로', '더', '와', '나', '전', '매우', '등', '내에', '을', '함'])

caselist = result['리콜사유'].values
text = ' '.join(caselist)
len(text)


font_path = './NanumGothic.ttf'
wc1 = WordCloud(max_font_size=200, stopwords=spwords, font_path=font_path,
                background_color='white', width=800, height=800)
wc1.generate(text)

plt.figure(figsize=(6, 4))
plt.imshow(wc1)
plt.tight_layout()
plt.axis('off')
plt.show()


In [ ]:
word_freq = wc1.process_text(text)
sorted_dict = sorted(word_freq.items(), key = lambda item: item[1], reverse = True)
print(sorted_dict)

In [ ]:
sorted_dict[0]

In [ ]:
print(f'{sorted_dict[0][0]}:{sorted_dict[0][1]}')
print(f'{sorted_dict[1][0]}:{sorted_dict[1][1]}')

In [ ]:
# 주요 원인인 EGR이 포함된 이유 상세 보기
egr = df[df['리콜사유'].str.contains('EGR')]
egr.head(5)

In [ ]:
len(egr)

In [ ]:
#EGR 원인으로 리콜된 가장 나쁜 모델3개는?
egr['차명'].value_counts().head(3)

```
리콜 주요 원인은? EGR 쿨러

그래서, 가장 나쁜 모델은? BMW 320d

```